In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from models.game_state import GameWorld
from models.generations import GameWorldBuilderAgentGeneration

llm = ChatOpenAI(temperature=0.7, model="gpt-4.1").with_structured_output(GameWorldBuilderAgentGeneration)

In [ ]:
import asyncio


messages = [
    SystemMessage(
        """
        You are generating a game world for a intractive narrative to take place within.
        Other LLM generations will iterate on this world, each controlling a single actor (character)
        by generating facts for them.

        The story is about is an active and escalating zombie outbreak.
        The setting is a suburban neighborhood.
        Generate at least five locations within this neighborhood.
            Locations should be things like a buidling interior, or the street, or a park, not a specific room
            Location ids are the name of the location in snake_case format
        Generate three survivor characters that are all strangers forced together. None of them are children.
            Generate interesting characters with unique traits. Do not discuss their determination or resolve or other emotions.
            The characters do not get along well with each other.
            Only provide outwardly observable facts.
            Actor ids are the full name (first and last name) of the character in snake_case format
        Generate at least two zombie characters that are mindless and will pursue and attack any non-zombie actor they encounter.
            These are aggressive, fast zombies, not slow ambling zombies. They will easily break through doors, windows, and barricades.
            These zombies will catch survivors if they are not disposed of quickly.
            Actor ids are the full name (first and last name) of the character in snake_case format. Even zombies have a first and last name.
        Generate some items.
            These may be weapons, tools, backpacks, or anything else that fits the setting.
            Make a few of them held by actors, and a few sitting out in locations
            Item ids are the qualified title of the item in snake_case format.
        Generate one active scenario involving generated characters.
        Generate some history within this scenario so that we are joining in media res.
            Note that there is a game loop that cycles through all actors in a scenario round-robin style.
            So the history within a scenario should reflect this with each actor in turn having outcomes resolved and then responding
        """),
    HumanMessage("Generate a game world according to the above instructions")
]

futures = [llm.ainvoke(messages) for i in range(1)]
responses = await asyncio.gather(*futures)

In [ ]:
import dill

generated = GameWorldBuilderAgentGeneration.model_validate(responses[0])
with open("generated_game_world.pkl", "wb") as f:
    dill.dump(generated, f)

In [ ]:

from models.game_state import Episode
from models.game_state import GameWorld
import dill

with open("generated_game_world.pkl", "rb") as f:
    generated = dill.load(f)
    
game_world = GameWorld()
game_world.actors = {actor.id: actor for actor in generated.actors}
game_world.locations = {location.id: location for location in generated.locations}
game_world.items = {item.id: item for item in generated.items}

for generated_ep in generated.active_episodes:
    episode = Episode()
    episode.actor_ids = generated_ep.actor_ids
    episode.event_history = generated_ep.event_history
    episode.actor_action_queues = {x.key: x.value for x in generated_ep.actor_action_queues}
    game_world.active_episodes.append(episode)

In [ ]:
import dill

with open("game_world.pkl", "wb") as f:
    dill.dump(game_world, f)

In [ ]:
from devtools import pprint

pprint(game_world)